In [1]:
import json

In [2]:
with open("E:\\merged-0-240_000.json", "r") as f:
    data = json.load(f)

In [3]:
import pandas as pd

In [4]:
df = pd.DataFrame(data)


In [5]:
df.head(5)

,t,s,id,d,c,cl,b,a
0,"Hatsun Food Company, Shri R.G. ... vs Commissi...","Customs, Excise and Gold Tribunal - Tamil Nadu",211998,"ORDER\n\n \n\n P.G. Chacko, Member (J)\n \n\n1...","[92590795, 30102403, 110162683, 30102403, 1146...",5,,NaN
1,Atar Singh & Ohters vs State Of U.P. & Another...,Allahabad High Court,212249,Court No. - 34\n\nCase :- FIRST APPEAL No. - 8...,[],0,,NaN
2,"Bharat vs State on 18 November, 2011",Gujarat High Court,212734,\n Gujarat High Court Case Information Syste...,[],0,,"Author: J.B.Pardiwala,"
3,Sajith N.L vs State Of Kerala Represented By T...,Kerala High Court,213164,\n\n \n\n \n\n \n \n IN THE HIGH COU...,[34253376],1,,NaN
4,Chandan Lal Dwivedi vs State Of U.P. And Other...,Allahabad High Court,213180,Court No. - 18\n\nCase :- WRIT - A No. - 41650...,[],0,,NaN


In [6]:
# Drop multiple columns
df.drop(['id', 'c', 'cl','b','a'], axis=1, inplace=True)

In [7]:
df.head(10)

,t,s,d
0,"Hatsun Food Company, Shri R.G. ... vs Commissi...","Customs, Excise and Gold Tribunal - Tamil Nadu","ORDER\n\n \n\n P.G. Chacko, Member (J)\n \n\n1..."
1,Atar Singh & Ohters vs State Of U.P. & Another...,Allahabad High Court,Court No. - 34\n\nCase :- FIRST APPEAL No. - 8...
2,"Bharat vs State on 18 November, 2011",Gujarat High Court,\n Gujarat High Court Case Information Syste...
3,Sajith N.L vs State Of Kerala Represented By T...,Kerala High Court,\n\n \n\n \n\n \n \n IN THE HIGH COU...
4,Chandan Lal Dwivedi vs State Of U.P. And Other...,Allahabad High Court,Court No. - 18\n\nCase :- WRIT - A No. - 41650...
5,Bali Alias Bahal Singh vs State Of Rajasthan o...,Rajasthan High Court,Name of accused- Convicted under sect...
6,[Section 30] [Complete Act],Central Government Act,
7,M/S.Padmavati Enterprises vs The Deputy Commis...,Madras High Court,...
8,[Section 30(1)] [Section 30] ...,Central Government Act,
9,Banarsi Das And Ors. vs Sagar Mal on 22 Februa...,Allahabad High Court,JUDGMENT\n \n\n1. This is an application purpo...


In [8]:
!pip install transformers


[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import re
import nltk
from transformers import DistilBertTokenizer

In [10]:
# Download NLTK resources (only need to do this once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Initialize WordNet lemmatizer and set of English stopwords
lemmatizer = nltk.stem.WordNetLemmatizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KEERTI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KEERTI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KEERTI\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
# Function to preprocess a single text
def preprocess_text(text):
    # Remove special characters and convert to lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    # Tokenize the text using DistilBERT tokenizer
    tokens = tokenizer.tokenize(text)
    # Remove stopwords, single characters, and perform lemmatization
    filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens 
                       if token.lower() not in stop_words and len(token) > 1]
    # Join the tokens back into a string
    preprocessed_text = ' '.join(filtered_tokens)
    return preprocessed_text


In [12]:
# Apply preprocessing to 't', 's', and 'd' columns in the DataFrame
df['t'] = df['t'].apply(preprocess_text)
df['s'] = df['s'].apply(preprocess_text)
df['d'] = df['d'].apply(preprocess_text)

# Tokenize and encode the preprocessed text
encoded_data = tokenizer(df['t'].tolist(), df['s'].tolist(), df['d'].tolist(), 
                         padding=True, truncation=True, return_tensors='pt')


In [14]:
print(encoded_data)

{'input_ids': tensor([[  101,  6045,  1001,  ...,     0,     0,     0],
        [  101, 29533,  1001,  ...,     0,     0,     0],
        [  101,  1001,  1001,  ...,     0,     0,     0],
        ...,
        [  101,  2930,  4029,  ...,     0,     0,     0],
        [  101,  5472,  1001,  ...,     0,     0,     0],
        [  101,  6294,  1001,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[  101,  2344, 18720,  ...,  3568, 12040,   102],
        [  101,  2457,  4090,  ...,     0,     0,     0],
        [  101, 14288,  2152,  ...,     0,     0,     0],
        ...,
        [  101,   102,     0,  ...,     0,     0,     0],
        [  101,  2152,  2457,  ...,     0,     0,     0],
        [  101,  8689,  6643,  ...,  2832,  4233,   102]])}


In [16]:
df.head(2)

,t,s,d
0,hat ##un food company shri ##g v commissioner ...,custom ex ##cise gold tribunal tamil nadu,order pg cha ##cko member appeal order commiss...
1,ata ##r singh oh ##ters v state another februa...,allah ##abad high court,court 34 case first appeal 81 ##8 1995 petitio...


In [19]:
print(encoded_data)

{'input_ids': tensor([[  101,  6045,  1001,  ...,     0,     0,     0],
        [  101, 29533,  1001,  ...,     0,     0,     0],
        [  101,  1001,  1001,  ...,     0,     0,     0],
        ...,
        [  101,  2930,  4029,  ...,     0,     0,     0],
        [  101,  5472,  1001,  ...,     0,     0,     0],
        [  101,  6294,  1001,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[  101,  2344, 18720,  ...,  3568, 12040,   102],
        [  101,  2457,  4090,  ...,     0,     0,     0],
        [  101, 14288,  2152,  ...,     0,     0,     0],
        ...,
        [  101,   102,     0,  ...,     0,     0,     0],
        [  101,  2152,  2457,  ...,     0,     0,     0],
        [  101,  8689,  6643,  ...,  2832,  4233,   102]])}


In [18]:
import pandas as pd

# Assuming 'encoded_data' is your DataFrame
df.to_csv("E:\\new.csv", index=False)